# Dataset prepration for Emory NLP Fine-tune

In [500]:
import os
import glob
import json
import random
from pathlib import Path
import pandas as pd # type: ignore

### Read data files

In [557]:
df_train = pd.read_csv("/Utilisateurs/umushtaq/emotion_analysis_comics/emory_nlp_FT/data_files/emorynlp_train_final.csv")
df_test = pd.read_csv("/Utilisateurs/umushtaq/emotion_analysis_comics/emory_nlp_FT/data_files/emorynlp_test_final.csv")
df_dev = pd.read_csv("/Utilisateurs/umushtaq/emotion_analysis_comics/emory_nlp_FT/data_files/emorynlp_dev_final.csv")

In [558]:
df_train = pd.concat([df_train, df_dev], axis=0).reset_index()

In [559]:
df_train

,index,Utterance,Speaker,Emotion,Scene_ID,Utterance_ID,Season,Episode,Start_Time,End_Time
0,0,"What you guys don't understand is, for us, kis...",['Monica Geller'],Joyful,1,1,1,2,00:00:02.877,00:00:07.548
1,1,"Yeah, right!.......Y'serious?",['Joey Tribbiani'],Neutral,1,2,1,2,00:00:04.504,00:00:07.548
2,2,"Oh, yeah!",['Phoebe Buffay'],Joyful,1,3,1,2,00:00:07.924,00:00:09.508
3,3,Everything you need to know is in that first k...,['Rachel Green'],Powerful,1,4,1,2,00:00:11.970,00:00:17.683
4,4,Absolutely.,['Monica Geller'],Powerful,1,5,1,2,00:00:14.139,00:00:15.097
...,...,...,...,...,...,...,...,...,...,...
8500,949,It's better! You can't go to a museum in your ...,['Joey Tribbiani'],Joyful,29,3,4,21,00:21:37.379,00:21:44.135
8501,950,"Well, You could, but... probably just the one ...",['Chandler Bing'],Joyful,29,4,4,21,00:21:45.220,00:21:47.555
8502,951,"I bet we could get videos of all the sites, ge...",['Joey Tribbiani'],Joyful,29,5,4,21,00:21:49.641,00:21:55.604
8503,952,"If we do that, we gotta get Die Hard.",['Chandler Bing'],Joyful,29,6,4,21,00:21:56.189,00:21:58.190


In [560]:
df_train.Emotion.value_counts()

Emotion
Neutral     2807
Joyful      1882
Scared      1068
Mad          882
Peaceful     720
Powerful     621
Sad          525
Name: count, dtype: int64

In [561]:
df_train.columns

Index(['index', 'Utterance', 'Speaker', 'Emotion', 'Scene_ID', 'Utterance_ID',
       'Season', 'Episode', 'Start_Time', 'End_Time'],
      dtype='object')

In [562]:
df_train[(df_train.Season == 1) & (df_train.Episode == 2)]


,index,Utterance,Speaker,Emotion,Scene_ID,Utterance_ID,Season,Episode,Start_Time,End_Time
0,0,"What you guys don't understand is, for us, kis...",['Monica Geller'],Joyful,1,1,1,2,00:00:02.877,00:00:07.548
1,1,"Yeah, right!.......Y'serious?",['Joey Tribbiani'],Neutral,1,2,1,2,00:00:04.504,00:00:07.548
2,2,"Oh, yeah!",['Phoebe Buffay'],Joyful,1,3,1,2,00:00:07.924,00:00:09.508
3,3,Everything you need to know is in that first k...,['Rachel Green'],Powerful,1,4,1,2,00:00:11.970,00:00:17.683
4,4,Absolutely.,['Monica Geller'],Powerful,1,5,1,2,00:00:14.139,00:00:15.097
...,...,...,...,...,...,...,...,...,...,...
120,120,No.,['Monica Geller'],Neutral,11,10,1,2,00:21:55.063,00:21:56.522
121,121,"You are, you're welling up.",['Ross Geller'],Powerful,11,11,1,2,00:21:56.732,00:22:00.443
122,122,Am not!,['Monica Geller'],Mad,11,12,1,2,00:21:58.191,00:22:00.443
123,123,You're gonna be an aunt.,['Ross Geller'],Joyful,11,13,1,2,00:22:00.652,00:22:03.779


In [563]:
scene_df_train = df_train.groupby(['Season', 'Episode', 'Scene_ID']).agg({
    'Utterance_ID': list,
    'Utterance': list,
    'Speaker': list,
    'Emotion': list
}).reset_index()

In [564]:
scene_df_train

,Season,Episode,Scene_ID,Utterance_ID,Utterance,Speaker,Emotion
0,1,2,1,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[What you guys don't understand is, for us, ki...","[['Monica Geller'], ['Joey Tribbiani'], ['Phoe...","[Joyful, Neutral, Joyful, Powerful, Powerful, ..."
1,1,2,2,"[1, 2, 3, 4, 5, 6, 8, 9, 12, 13, 14, 15, 16, 1...","[No, it's good, it is good, it's just that- mm...","[['Ross Geller'], ['Marsha'], ['Ross Geller'],...","[Scared, Sad, Scared, Mad, Neutral, Mad, Scare..."
2,1,2,6,"[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...","[Sorry I'm late, I was stuck at work. There wa...","[['Ross Geller'], ['Susan Bunch'], ['Carol Wil...","[Neutral, Neutral, Neutral, Mad, Neutral, Neut..."
3,1,2,7,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13]","[Barry?, C'mon in., Are you sure?, Yeah! It's ...","[['Rachel Green'], ['Barry Farber'], ['Rachel ...","[Neutral, Neutral, Scared, Neutral, Scared, Pe..."
4,1,2,8,"[2, 3, 4, 5, 6, 7, 8, 10, 11]","[Give me a 'for instance'., Well, uh, uh, I do...","[['Carol Willick'], ['Ross Geller'], ['Carol W...","[Mad, Scared, Powerful, Mad, Powerful, Mad, Ne..."
...,...,...,...,...,...,...,...
743,4,24,13,"[2, 8, 10, 12, 13, 16, 17]","[Hello?, Ohh great!!!, But, he loves Emily?, O...","[['Joey Tribbiani'], ['Joey Tribbiani'], ['Joe...","[Peaceful, Joyful, Joyful, Powerful, Peaceful,..."
744,4,24,14,"[4, 8, 9]","[Ehh, pardon me?, Well I-I think your wrong., ...","[['Rachel Green'], ['Rachel Green'], ['Passeng...","[Mad, Peaceful, Mad]"
745,4,24,15,"[2, 4, 5, 6, 7, 8, 9, 11]","[Do something..... Something...., Hey., Hey., ...","[['Joey Tribbiani'], ['Joey Tribbiani'], ['Cha...","[Neutral, Neutral, Neutral, Neutral, Mad, Scar..."
746,4,24,16,"[3, 5, 9, 11, 13]","[I know, it's horrible isn't it?, I just hope....","[['Andrea Waltham'], ['Judy Geller'], ['Joey T...","[Powerful, Joyful, Neutral, Scared, Scared]"


In [581]:
list_lengths = scene_df_train['Utterance'].apply(len)

# Calculate min, max, and mean of the lengths
min_length = list_lengths.min()
max_length = list_lengths.max()
mean_length = list_lengths.mean()

In [584]:
mean_length

np.float64(11.370320855614974)

In [565]:
scene_df_test = df_test.groupby(['Season', 'Episode', 'Scene_ID']).agg({
    'Utterance_ID': list,
    'Utterance': list,
    'Speaker': list,
    'Emotion': list
}).reset_index()

In [587]:
scene_df_test

,Season,Episode,Scene_ID,Utterance_ID,Utterance,Speaker,Emotion
0,1,1,4,"[1, 3, 4, 5, 7, 8, 10, 11, 12, 14, 16, 17, 18,...",[I'm supposed to attach a brackety thing to th...,"[['Ross Geller'], ['Joey Tribbiani'], ['Chandl...","[Mad, Neutral, Joyful, Neutral, Neutral, Scare..."
1,1,1,5,"[1, 5, 6, 8, 9]","[Oh my God!, -leg?, That's one way! Me, I- I w...","[['Monica Geller'], ['Monica Geller'], ['Paul ...","[Mad, Joyful, Joyful, Peaceful, Powerful]"
2,1,1,7,"[1, 2, 3, 4, 9]","[I'm divorced! I'm only 26 and I'm divorced!, ...","[['Ross Geller'], ['Joey Tribbiani'], ['Chandl...","[Sad, Mad, Mad, Sad, Mad]"
3,1,1,8,"[5, 9, 11, 14]","[Isn't there?, It's okay..., Two years., ...Ye...","[['Paul the Wine Guy'], ['Paul the Wine Guy'],...","[Scared, Sad, Neutral, Neutral]"
4,1,1,10,[3],"[Angela's the screamer, Andrea has cats.]",[['Chandler Bing']],[Joyful]
...,...,...,...,...,...,...,...
74,4,20,6,"[2, 5, 6, 7, 8, 9, 10, 11, 12]",[Does she use the cups? Yes! I believe she doe...,"[['Monica Geller'], ['Phoebe Buffay'], ['Monic...","[Powerful, Joyful, Scared, Neutral, Scared, Ma..."
75,4,20,8,"[2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 16, 18, 19]","[Hey, Rachel., Hi!, What's up? You're voice so...","[['Joshua Burgin'], ['Rachel Green'], ['Joshua...","[Neutral, Neutral, Peaceful, Peaceful, Powerfu..."
76,4,20,9,"[2, 3, 4, 5, 6, 8, 9, 11, 12, 13, 14, 15, 16, ...","[Your name, please?, Joey Tribbiani., Um-hmm, ...","[['Sleep Clinic Worker'], ['Joey Tribbiani'], ...","[Peaceful, Joyful, Powerful, Neutral, Powerful..."
77,4,20,10,"[2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, ...","[Okay, ready?, Yeah., Okay., I got it! Mine!, ...","[['Monica Geller'], ['Phoebe Buffay'], ['Monic...","[Neutral, Neutral, Neutral, Joyful, Joyful, Jo..."


### Paths

In [537]:
current_dir = Path.cwd()
dataset_dir = current_dir / "emotion_analysis_comics" / "emory_nlp_FT" / "datasets"

In [538]:
dataset_dir

PosixPath('/Utilisateurs/umushtaq/emotion_analysis_comics/emory_nlp_FT/datasets')

### Prepare prompts

In [539]:
# Formatting Fx
# Build questoin
# Build answer

In [593]:
def formatting_fct(instruction="", input="", output=""):
    
    prompt_d ={
            
        "instruction": f"""{instruction}""",
        "input": f"""{input}""",
        "output": f"""{output}"""
            
        }
    
    return prompt_d

In [594]:
def build_instruction():
    
    emotion_classes = ["Mad", "Scared", "Sad", "Powerful", "Peaceful", "Joyful", "Neutral"]
    formatted_classes = ", ".join([f'"{emotion}"' for emotion in emotion_classes])

    instruction = f"""### You are an expert in Emotion Analysis for the Friends TV show.

You are given a scene from a Friends episode containing multiple utterances and the speaker for each utterance.

Your task is to classify each utterance in the scene with ONLY ONE emotion class from these options: "Mad", "Scared", "Sad", "Powerful", "Peaceful", "Joyful", or "Neutral".

Output Instructions:
1. Return ONLY a JSON object with emotion classifications for all utterances in the scene
2. The JSON must have this exact structure: {{"emotions": [emotion1, emotion2, ... ]}}
3. EMOTION must be one of the specified emotion classes
4. Do NOT include any additional text or explanation
5. Identify only one applicable emotion for each utterance only from the following classes:
   {formatted_classes}

"""    

    return instruction

In [595]:
# def build_instruction():
    
#     emotion_classes = ["Mad", "Scared", "Sad", "Powerful", "Peaceful", "Joyful", "Neutral"]
#     formatted_classes = ", ".join([f'"{emotion}"' for emotion in emotion_classes])

#     instruction = f"""### You are an expert in Emotion Analysis for the Friends TV show.

# You are given an utterance from a Friends episode.

# Your task is to classify the utterance with a single emotion class from these options: "Mad", "Scared", "Sad", "Powerful", "Peaceful", "Joyful", or "Neutral".

# Output Instructions:
# 1. Return ONLY a JSON object with a single emotion class
# 2. The JSON must have this exact structure: {{"emotion_class": "EMOTION"}}
# 3. EMOTION must be one of the specified emotion classes
# 4. Do NOT include any additional text or explanation
# 5. Identify only one applicable emotions only from the following classes:
#    {formatted_classes}

# """    

#     return instruction

In [596]:
def build_scene_utterances(utterance_ids, utterances, speakers):
    
    #[utterance_id] [utterance] [speaker]
    
    #result_string = "\n".join(f"{uid}: {speaker} - {utt}" for uid, utt, speaker in zip(utterance_ids, utterances, speakers))
    
    #esult_string = "\n".join(f"Utterance {uid}: {utt} - Speaker: {speaker}" for uid, utt, speaker in zip(utterance_ids, utterances, speakers))
    
    result_string = "\n".join(f"Utterance {i+1}: {utt} - Speaker: {speaker}" for i, (utt, speaker) in enumerate(zip(utterances, speakers)))


    question = f"""Now classify these utterances in the scene:\n{result_string}"""

    return question

In [597]:
# utterance_emotions_l = []
#     emotion_class_labels = ["Anger", "Disgust", "Fear", "Sadness", "Surprise", "Joy"]

#     if utterance_emotions == 'Neutral':
        
#         utterance_emotions_l.append([utterance_emotions])
    
#     else:
#         utterance_emotions = utterance_emotions.split("-")
       
#         #emotion_annotation_l = []

#         for idx, emotion_annotation in enumerate(utterance_emotions):

#             if '0' not in emotion_annotation:
         
#                 #emotion_annotation_l.append(emotion_class_labels[idx])
#                 utterance_emotions_l.append(emotion_annotation[:-1])
            
#         #title_emotions_l.append(emotion_annotation_l)

In [598]:
def build_answer(emotions):    
                

    return json.dumps({"emotions": emotions})

### Build Data Files

In [599]:
#df_train = df[df.split == 'TRAIN'].reset_index()

data_file_train = []

for _, row in scene_df_train.iterrows():

    instruction = build_instruction()
    question = build_scene_utterances(row.Utterance_ID, row.Utterance, row.Speaker)
    answer = build_answer(row.Emotion)
    
    data_file_train.append( formatting_fct(instruction, question, answer) )

In [600]:
len(data_file_train)

748

In [601]:
for i in range(5):
    print(data_file_train[i])

{'instruction': '### You are an expert in Emotion Analysis for the Friends TV show.\n\nYou are given a scene from a Friends episode containing multiple utterances and the speaker for each utterance.\n\nYour task is to classify each utterance in the scene with ONLY ONE emotion class from these options: "Mad", "Scared", "Sad", "Powerful", "Peaceful", "Joyful", or "Neutral".\n\nOutput Instructions:\n1. Return ONLY a JSON object with emotion classifications for all utterances in the scene\n2. The JSON must have this exact structure: {"emotions": [emotion1, emotion2, ... ]}\n3. EMOTION must be one of the specified emotion classes\n4. Do NOT include any additional text or explanation\n5. Identify only one applicable emotion for each utterance only from the following classes:\n   "Mad", "Scared", "Sad", "Powerful", "Peaceful", "Joyful", "Neutral"\n\n', 'input': "Now classify these utterances in the scene:\nUtterance 1: What you guys don't understand is, for us, kissing is as important as any 

In [602]:
print(data_file_train[1]['instruction'])

### You are an expert in Emotion Analysis for the Friends TV show.

You are given a scene from a Friends episode containing multiple utterances and the speaker for each utterance.

Your task is to classify each utterance in the scene with ONLY ONE emotion class from these options: "Mad", "Scared", "Sad", "Powerful", "Peaceful", "Joyful", or "Neutral".

Output Instructions:
1. Return ONLY a JSON object with emotion classifications for all utterances in the scene
2. The JSON must have this exact structure: {"emotions": [emotion1, emotion2, ... ]}
3. EMOTION must be one of the specified emotion classes
4. Do NOT include any additional text or explanation
5. Identify only one applicable emotion for each utterance only from the following classes:
   "Mad", "Scared", "Sad", "Powerful", "Peaceful", "Joyful", "Neutral"




In [603]:
print(data_file_train[1]['input'])

Now classify these utterances in the scene:
Utterance 1: No, it's good, it is good, it's just that- mm- doesn't she seem a little angry? - Speaker: ['Ross Geller']
Utterance 2: Well, she has issues. - Speaker: ['Marsha']
Utterance 3: Does she. - Speaker: ['Ross Geller']
Utterance 4: He's out banging other women over the head with a club, while she sits at home trying to get the mastodon smell out of the carpet! - Speaker: ['Marsha']
Utterance 5: Marsha, these are cave people. Okay? They have issues like 'Gee, that glacier's getting kinda close.' See? - Speaker: ['Ross Geller']
Utterance 6: Speaking of issues, isn't that your ex-wife? - Speaker: ['Marsha']
Utterance 7: No. No. - Speaker: ['Ross Geller']
Utterance 8: Yes, it is. Carol! Hi! - Speaker: ['Marsha']
Utterance 9: Hi. - Speaker: ['Ross Geller']
Utterance 10: So. - Speaker: ['Carol Willick']
Utterance 11: You look great. I, uh... I hate that. - Speaker: ['Ross Geller']
Utterance 12: Sorry. You look good too. - Speaker: ['Carol W

In [604]:
print(data_file_train[1]['output'])

{"emotions": ["Scared", "Sad", "Scared", "Mad", "Neutral", "Mad", "Scared", "Joyful", "Neutral", "Neutral", "Scared", "Joyful", "Joyful", "Neutral", "Peaceful", "Mad", "Scared", "Neutral", "Scared"]}


In [605]:
#df_test = df[df.split == 'TEST'].reset_index()

data_file_test = []

for _, row in scene_df_test.iterrows():

    instruction = build_instruction()
    question = build_scene_utterances(row.Utterance_ID, row.Utterance, row.Speaker)
    answer = build_answer(row.Emotion)
    
    data_file_test.append( formatting_fct(instruction, question, answer) )

In [606]:
len(data_file_test)

79

In [607]:
for i in range(5):
    print(data_file_test[i])

{'instruction': '### You are an expert in Emotion Analysis for the Friends TV show.\n\nYou are given a scene from a Friends episode containing multiple utterances and the speaker for each utterance.\n\nYour task is to classify each utterance in the scene with ONLY ONE emotion class from these options: "Mad", "Scared", "Sad", "Powerful", "Peaceful", "Joyful", or "Neutral".\n\nOutput Instructions:\n1. Return ONLY a JSON object with emotion classifications for all utterances in the scene\n2. The JSON must have this exact structure: {"emotions": [emotion1, emotion2, ... ]}\n3. EMOTION must be one of the specified emotion classes\n4. Do NOT include any additional text or explanation\n5. Identify only one applicable emotion for each utterance only from the following classes:\n   "Mad", "Scared", "Sad", "Powerful", "Peaceful", "Joyful", "Neutral"\n\n', 'input': "Now classify these utterances in the scene:\nUtterance 1: I'm supposed to attach a brackety thing to the side things, using a bunch 

In [608]:
# #df_test = df[df.split == 'TEST'].reset_index()

# data_file_dev = []

# for index, _ in df_dev.iterrows():
    
#     i = index

#     instruction = build_instruction()
#     question = build_tagged_text(df_dev.iloc[i].Utterance)
#     answer = build_answer(df_dev.iloc[i].Emotion)
    
#     data_file_dev.append( formatting_fct(instruction, question, answer) )

In [609]:
#len(data_file_dev)

In [610]:
# for i in range(5):
#     print(data_file_dev[i])

### Create and save JSON files

In [611]:
dataset_dir.mkdir(parents=True, exist_ok=True)

In [612]:
file_path = Path(dataset_dir) / "emorynlp_utterance_scene_train.json"

with open(file_path, 'w') as file:
    
    json.dump(data_file_train, file)

In [613]:
file_path = Path(dataset_dir) / "emorynlp_utterance_scene_test.json"

with open(file_path, 'w') as file:
    
    json.dump(data_file_test, file)

In [ ]:
# file_path = Path(dataset_dir) / "emorynlp_utterance_p2_dev.json"

# with open(file_path, 'w') as file:
    
#     json.dump(data_file_dev, file)